In [45]:
import requests
import json
import pandas as pd

with open('api_key.txt') as f:
    api = f.read()

headers= {'X-API-Key': api}

In [46]:
# get list of sensors within lat/long bounding box (defined by NW/SE corners)
# 
# NWlng = -98
# NWlat = 30.5
# 
# SElng = -97.5
# SElat = 30
# 
# url = f'https://api.purpleair.com/v1/sensors?fields=latitude,longitude,altitude&location_type=0&modified_since=0&max_age=0&nwlng={NWlng}&nwlat={NWlat}&selng={SElng}&selat={SElat}'
# 
# response = requests.get(url, headers=headers)
# print("Status Code: ", response.status_code)
# 
# content = json.loads(response.content)
# data = content["data"]
# columns = content["fields"]
# 
# df = pd.DataFrame(data, columns=columns)
# df

Status Code:  200


,sensor_index,latitude,longitude,altitude
0,2796,30.276873,-97.837310,726
1,2806,30.483000,-97.848500,984
2,2862,30.366156,-97.765290,899
3,4041,30.306090,-97.726930,618
4,14983,30.130820,-97.844330,662
...,...,...,...,...
186,213765,30.307280,-97.826090,828
187,213789,30.228407,-97.754230,664
188,222223,30.308128,-97.801890,611
189,236401,30.337454,-97.711580,712


In [47]:
# df.to_csv('austin_sensors.csv', index=False)

In [48]:
sensors = pd.read_csv('austin_sensors.csv')['sensor_index']
# Should be (191,)
sensors.shape

(191,)

In [30]:
# For testing
# sensors = sensors.iloc[:1]

In [41]:
# pm2.5, humidity, temp, pressure
fields = ['humidity', 'temperature', 'pressure', 'pm2.5_alt', 'pm2.5_atm']
# UTC time
start_timestamp = '2024-11-09T06:00:00.000Z'
end_timestamp = '2024-11-10T06:00:00.000Z'
# Average over 30-min intervals
average = 30

df = pd.DataFrame()
for si in sensors:
    url = f'https://api.purpleair.com/v1/sensors/{si}/history?average={average}&start_timestamp={start_timestamp}&end_timestamp={end_timestamp}&fields={",".join(fields)}'
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f'Pulling data for sensor {si} failed')
        continue
    content = json.loads(response.content)
    data = content["data"]
    columns = content["fields"]
    new_df = pd.DataFrame(data, columns=columns)
    new_df['sensor_index'] = content['sensor_index']
    if new_df.shape[0] > 0:
        df = pd.concat([df, new_df])

C:\Users\Victor Nguyen\AppData\Local\Temp\ipykernel_13176\2286905578.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df])
C:\Users\Victor Nguyen\AppData\Local\Temp\ipykernel_13176\2286905578.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df])
C:\Users\Victor Nguyen\AppData\Local\Temp\ipykernel_13176\2286905578.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, th

In [25]:
df.to_csv('purpleair_data.csv', index=False)

In [42]:
df

,time_stamp,humidity,temperature,pressure,pm2.5_alt,pm2.5_atm,sensor_index
0,2024-11-10T02:30:00Z,46.533,71.400,987.641,1.9,2.4970,2796
1,2024-11-10T01:30:00Z,42.867,73.467,987.401,3.3,5.1585,2796
2,2024-11-09T16:30:00Z,27.467,81.467,989.887,0.4,0.0075,2796
3,2024-11-09T07:00:00Z,48.200,72.000,989.408,1.6,1.7990,2796
4,2024-11-09T13:30:00Z,43.533,73.133,990.133,0.6,0.0995,2796
...,...,...,...,...,...,...,...
43,2024-11-09T10:00:00Z,34.467,71.267,993.003,0.4,0.0085,244811
44,2024-11-10T02:00:00Z,30.067,76.467,991.444,6.5,10.3985,244811
45,2024-11-09T17:30:00Z,21.000,79.800,993.109,0.4,0.0170,244811
46,2024-11-09T17:00:00Z,22.200,77.400,993.474,0.4,0.0165,244811
